In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Softmax, Input, BatchNormalization, Dropout

from sklearn.model_selection import train_test_split

import tensorflow as tf
from load_policy import load_policy
import pickle
import numpy as np
import gym

import tf_util


Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

In [3]:
expert_policy_file = "experts/Reacher-v1.pkl"
env_name = "Reacher-v1"
rounds = 1000
max_steps = 1000

In [4]:
data = pickle.load(open(expert_policy_file, "rb"))
policy_fn = load_policy(expert_policy_file)

obs (1, 11) (1, 11)


In [5]:
observations = []
actions = []
rewards = []

with tf.Session() as sess:
#     tf_util.initialize()
    env = gym.make(env_name)
    for i in range(rounds):
        obs = env.reset()
        r = 0
        for s in range(max_steps):
            action = policy_fn(obs[None,:])
            observations.append(obs)
            actions.append(action)
            obs, r_, done, _ = env.step(action)
            rewards.append(r_)
            r += r_
            if done:
                break
        if i and i%100 == 0:
            print(i)
            

[2018-06-06 14:08:21,196] Making new env: Reacher-v1


100
200
300
400
500
600
700
800
900


In [6]:
observations = np.array(observations)
actions = np.vstack(actions)
rewards = np.array(rewards)

obs_train, obs_valid, act_train, act_valid = \
    train_test_split(observations, actions, test_size = 0.2, random_state = 1998)

In [7]:
actions.shape

(1000000, 2)

In [33]:
input_dim = observations.shape[-1]
bc = Sequential()
bc.add(Dense(32, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
bc.add(Dense(512, input_dim=input_dim, kernel_initializer='normal', activation='relu'))

# bc.add(Dense(512, kernel_initializer='normal', activation='relu'))
# bc.add(BatchNormalization())
# bc.add(Dropout(0.5))
bc.add(Dense(32, kernel_initializer='normal', activation='relu'))

bc.add(Dense(2, kernel_initializer='normal'))


In [34]:
bc.compile(optimizer='adam',loss='mse', metrics=['accuracy', 'mae'])

In [35]:
bc.fit(obs_train, act_train, validation_data=[obs_valid, act_valid],  epochs=10, batch_size= 512)

Train on 800000 samples, validate on 200000 samples
Epoch 1/10
800000/800000 [==============================] - 3s 4us/step - loss: 1.0037e-04 - acc: 0.5459 - mean_absolute_error: 0.0030 - val_loss: 2.5185e-05 - val_acc: 0.5542 - val_mean_absolute_error: 0.0018
Epoch 2/10
800000/800000 [==============================] - 3s 4us/step - loss: 1.9726e-05 - acc: 0.5557 - mean_absolute_error: 0.0016 - val_loss: 1.6545e-05 - val_acc: 0.5392 - val_mean_absolute_error: 0.0023
Epoch 3/10
800000/800000 [==============================] - 3s 4us/step - loss: 1.0994e-05 - acc: 0.5584 - mean_absolute_error: 0.0013 - val_loss: 1.2519e-05 - val_acc: 0.5381 - val_mean_absolute_error: 0.0018
Epoch 4/10
800000/800000 [==============================] - 3s 4us/step - loss: 8.3437e-06 - acc: 0.5591 - mean_absolute_error: 0.0012 - val_loss: 6.8048e-06 - val_acc: 0.5438 - val_mean_absolute_error: 0.0010
Epoch 5/10
800000/800000 [==============================] - 3s 4us/step - loss: 6.2090e-06 - acc: 0.5602 - m

In [11]:
rewards.sum()/1000

-22.77505284176671

In [20]:
o = np.random.random(11)[None, :]
bc.predict(o)

array([[-0.30525178, -0.2506469 ]], dtype=float32)

In [21]:
with tf.Session():
    print(policy_fn(o))

[[0.8586951 1.2538567]]


In [36]:
m_rewards = []

def run_model(m, m_rounds, max_steps):
    env = gym.make(env_name)
    for i in range(m_rounds):
        obs = env.reset()
        r = 0
        for s in range(max_steps):
            action = m.predict(obs[None,:])
#             observations.append(obs)
#             actions.append(action)
            obs, r_, done, _ = env.step(action)
            m_rewards.append(r_)
#             r += r_
            if done:
                break

In [37]:
mr = 1000
mstep = 1000
run_model(bc, mr, mstep)

[2018-06-06 18:02:22,234] Making new env: Reacher-v1


In [38]:
sum(m_rewards)/mr

-32.649625237324535

In [39]:
len(m_rewards)

1000000

In [30]:
2

2